In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
dataset_path = 'Datasets/IndianRecipeData.xlsx'

indian_recipe_data = pd.read_excel(dataset_path)

print(indian_recipe_data.head())


In [ ]:
# Drop duplicates if any
indian_recipe_data.drop_duplicates(inplace=True)

# Handle missing values if needed
indian_recipe_data.dropna(inplace=True)



In [ ]:
column_names = indian_recipe_data.columns

# Print the column names
print(column_names)

In [ ]:
import pandas as pd

# Load your dataset
dataset_path = 'Datasets/IndianRecipeData.xlsx'
indian_recipe_data = pd.read_excel(dataset_path)

# Check if the "Course" column exists in the dataset
if 'Course' in indian_recipe_data.columns:
    # Get the unique entries in the "Course" column
    course_labels = indian_recipe_data['Course'].unique()

    # Print the unique course labels
    print("Unique Course Labels:")
    for label in course_labels:
        print(label)
else:
    print("The dataset does not contain a 'Course' column.")

In [ ]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Define your dataset path
dataset_path = 'Datasets/IndianRecipeData.xlsx'
indian_recipe_data = pd.read_excel(dataset_path)


#--------------------------------------input ingredients, course and time-------------------------------------
sample_user_input = "ragi"
sample_course_input = "satvic"
sample_max_time_input = 30  # in minutes
#----------------------------------------------------------------------------------------------------

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    else:
        return ''  # Return empty string for non-string inputs

def calculate_similarity(user_ingredients, recipe_ingredients, vectorizer):
    user_vector = vectorizer.transform([user_ingredients])
    recipe_vector = vectorizer.transform([recipe_ingredients])
    similarity_score = cosine_similarity(user_vector, recipe_vector)[0][0]
    return similarity_score

def recommend_recipes(user_ingredients, recipe_data, vectorizer):
    priority_recipes = []
    other_recipes = []

    # Split the user input ingredients into a list
    user_ingredients_list = user_ingredients.split(", ")

    for index, row in recipe_data.iterrows():
        recipe_ingredients = preprocess_text(row['Ingredients'])
        similarity_score_name = calculate_similarity(user_ingredients, preprocess_text(row['RecipeName']), vectorizer)
        similarity_score_translated = calculate_similarity(user_ingredients, preprocess_text(row['TranslatedRecipeName']), vectorizer)
        similarity_score_ingredients = calculate_similarity(user_ingredients, recipe_ingredients, vectorizer)

        # Check if all user ingredients are in the recipe ingredients
        if all(ingredient.strip() in recipe_ingredients.split(", ") for ingredient in user_ingredients_list):
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], 1.0))  # Set similarity score to 1 for exact matches
        # Check if the entered ingredient is in the recipe name as a major element
        elif similarity_score_name > 0.5:
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], similarity_score_name))
        # Check if the entered ingredient is in the translated recipe name as a major element
        elif similarity_score_translated > 0.5:
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], similarity_score_translated))
        # Check if the entered ingredient is in the recipe ingredients
        elif similarity_score_ingredients > 0.5:
            priority_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], similarity_score_ingredients))
        else:
            other_recipes.append((row['RecipeName'], row['Ingredients'], row['Instructions'], 0))  # Set a low similarity score for others

    priority_recipes.sort(key=lambda x: x[3], reverse=True)
    other_recipes.sort(key=lambda x: x[3], reverse=True)

    return priority_recipes, other_recipes



def generate_sequence_of_recommendations(priority_recipes, other_recipes):
    sequence_recommendations = []
    item_number = 1

    for recipe, _, _, _ in priority_recipes:
        sequence_recommendations.append(f"{item_number}. {recipe}")
        item_number += 1

    for recipe, _, _, _ in other_recipes:
        sequence_recommendations.append(f"{item_number}. {recipe}")
        item_number += 1

    return sequence_recommendations

def recommendation_system(user_input, course_input, max_time_input, dataset):
    user_input_processed = preprocess_text(user_input)
    dataset['Ingredients'] = dataset['Ingredients'].apply(preprocess_text)

    vectorizer = TfidfVectorizer()
    vectorizer.fit(dataset['Ingredients'])

    priority_recipes, other_recipes = recommend_recipes(user_input_processed, dataset, vectorizer)
    sequence_recommendations = generate_sequence_of_recommendations(priority_recipes, other_recipes)

    return sequence_recommendations, priority_recipes

# Call the recommendation_system function with the sample input and your dataset
sequence_recommendations, priority_recipes = recommendation_system(sample_user_input, sample_course_input, sample_max_time_input, indian_recipe_data)

# Print the recommended recipes
print("Recommended Recipes:")
for item in sequence_recommendations:
    print(item)

In [ ]:
from IPython.display import display, Markdown

#--------------------------- input item no.--------------------------------------
selected_item_number = 2
#--------------------------------------------------------------------------------

# Check if the selected item number is valid
if 1 <= selected_item_number <= len(sequence_recommendations):
    selected_item_str = sequence_recommendations[selected_item_number - 1]  # Get the item string
    selected_item_name = selected_item_str.split(". ")[1]  # Extract recipe name

    # Find the recipe details in the dataset
    selected_recipe_details = indian_recipe_data[indian_recipe_data['RecipeName'] == selected_item_name]

    if not selected_recipe_details.empty:
        selected_recipe_details = selected_recipe_details.iloc[0]  # Take the first row (assuming unique recipe names)
        display(Markdown("### Recipe for Selected Item:"))
        display(Markdown(f"**Recipe Name:** {selected_recipe_details['RecipeName']}"))
        display(Markdown(f"**Ingredients:**\n{selected_recipe_details['Ingredients']}"))
        display(Markdown(f"**Instructions:**\n{selected_recipe_details['Instructions']}"))
    else:
        print("Recipe details not found.")
else:
    print("Invalid item number. Please enter a valid number.")